# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
#install scikit-learn==0.22.1
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet') # download for lemmatization
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import numpy as np
from nltk.stem.wordnet import WordNetLemmatizer
from sqlalchemy import create_engine
import pandas as pd
from itertools import chain

from sklearn.datasets import make_multilabel_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputClassifier





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///Anand_ETL_Pipeline_16022020.db')
df = pd.read_sql_table('Cyclone',engine)

X = df['message'].values
y = df.iloc[:,4:-1]

print(X.size,y.size)
y.shape

26216 917560


(26216, 35)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    words = []
    tokens = []

    #normalize
    for i in range(len(text)):
        if (type(text[i]).__name__ == 'list'):
            for j in range(len(text[i])):
                words.append([re.sub(r"[^a-zA-Z]", " ", text[i][j].lower())])
        else:         
            words.append(re.sub(r"[^a-zA-Z]", " ", text[i].lower()))

            
            
    #tokenize        
    for i in range(len(words)):
        if (type(words[i]).__name__ == 'list'):
            for j in range(len(words[i])):
                tokens.append(word_tokenize(words[i][j]))
        else:         
            tokens.append(word_tokenize(words[i]))
            
    
    # putting all tokens in one list rather than leaving them list of lists
    tokens = list(chain(*tokens))
    
    #remove stop words
    tokens = [t for t in tokens if t not in stopwords.words("english")]
    
    #remove duplicate tokens from the list
    #tokens = list(set(tokens))
    
    #lemmatize
    # Reduce words to their root form
    lemmed_before = [WordNetLemmatizer().lemmatize(w) for w in tokens]
    
    # Lemmatize verbs by specifying pos
    lemmed_after = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed_before]

    return lemmed_after

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# build pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
#Spliting into Training and Test data sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

###using 1% of actual training set for troubleshooting
#X_train = X_train[:len(X_train)/100]
#y_train = y_train[:len(y_train)/100]

# train classifier
pipeline.fit(X_train,y_train)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(17564,) (17564, 35) (8652,) (8652, 35)


In [6]:
y_pred = pipeline.predict(X_test)

# 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
print(y_pred.shape,y_test.shape)

(8652, 35) (8652, 35)


In [8]:
import pandas
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

target_names = ['class 0', 'class 1', 'class 2']

metrics = []
#print(y_test)
for i, col in enumerate(y_test):
    report = classification_report(y_test[col],y_pred[:,i],target_names=target_names)
    #print(y_test[:i],y_pred[:,i])
    #print(col , report)
    metrics.append(report)


metrics = np.array(metrics)
#print(metrics)

#metrics_df = pd.DataFrame(data = metrics,  index=None, columns = ['Precision', 'Recall', 'F1', 'Support'])
#print(metrics_df)



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 3
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 3
  .format(len(labels), len(target_names))


In [9]:
#####======#########
pprint(pipeline.get_params())


{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1),
 'clf__estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
 'clf__estimator__bootstrap': True,
 'clf__estimator__class_weight': None,


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'vect__min_df': [1],
    'vect__max_df': [1.0],
    'tfidf__norm': ['l1','l2']
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = pipeline, param_grid = param_grid,cv = 3, n_jobs = -1, verbose = 2)





#from sklearn.model_selection import GridSearchCV
#from IPython.core.debugger import set_trace
#rf = RandomForestRegressor()

#pprint(rf.get_params())


# specify parameters for grid search
#parameters = {}

# Instantiate the grid search model
#cv = GridSearchCV(rf, param_grid=parameters,refit=False,return_train_score=False)

#pprint(cv)

#print(type(X_train))
#set_trace()
grid_search.fit(X_train, y_train)
#set_trace()

#cv.best_params_
#y_pred2 = cv.predict(X_test)

#print(y_pred2)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] tfidf__norm=l1, vect__max_df=1.0, vect__min_df=1 ................
[CV] . tfidf__norm=l1, vect__max_df=1.0, vect__min_df=1, total= 9.3min
[CV] tfidf__norm=l1, vect__max_df=1.0, vect__min_df=1 ................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 15.2min remaining:    0.0s


[CV] . tfidf__norm=l1, vect__max_df=1.0, vect__min_df=1, total= 9.3min
[CV] tfidf__norm=l1, vect__max_df=1.0, vect__min_df=1 ................



### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!



In [ ]:
### Test your model

from sklearn.preprocessing import label_binarize

# Use label_binarize to be multi-label like settings
Y = label_binarize(y, classes=[0, 1, 2])
n_classes = Y.shape[1]

# Split into training and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.5,
                                                    random_state=random_state)

# We use OneVsRestClassifier for multi-label prediction
from sklearn.multiclass import OneVsRestClassifier

# Run classifier
classifier = OneVsRestClassifier(svm.LinearSVC(random_state=random_state))
classifier.fit(X_train, Y_train)
y_score = classifier.decision_function(X_test)


from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

# For each class
precision = dict()
recall = dict()
average_precision = dict()
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(Y_test[:, i],
                                                        y_score[:, i])
    average_precision[i] = average_precision_score(Y_test[:, i], y_score[:, i])

# A "micro-average": quantifying score on all classes jointly
precision["micro"], recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
    y_score.ravel())
average_precision["micro"] = average_precision_score(Y_test, y_score,
                                                     average="micro")
print('Average precision score, micro-averaged over all classes: {0:0.2f}'
      .format(average_precision["micro"]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
### Export your model as a pickle file
import pickle

# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.